In [30]:
## from prompt_templates import kshot_prompt_templates

from finetuning.prepare_data import add_observation_to_raw_finetuning_dataset

from utils.utils import get_models_metadata, prepare_chat_history, get_default_args

from parse import parse

%load_ext autoreload
%autoreload 2

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
models_metadata = get_models_metadata()

# ask how it works

In [3]:
kshot_ask_how_it_works_prompt = kshot_prompt_templates.ask_how_it_works

kshot_ask_how_it_works_prompt = kshot_ask_how_it_works_prompt.split("Below are some examples:")[-1]

separated_observations = kshot_ask_how_it_works_prompt.split("Current section chat:")[1:]

for i in range(len(separated_observations)):
    separated_observations[i] = "Current section chat:" + separated_observations[i].rstrip()

separated_observations[-1] = separated_observations[-1][:-6]

separated_observations

In [9]:
len(separated_observations)

34

In [14]:
all_observation_details = []
for i in range(len(separated_observations)):
    print(i)
    prompt = separated_observations[i] \
        .rsplit("Interviewer: ", maxsplit=1)[0] \
        .replace("\n\n\n", "\n{}\n\n".format(get_default_args(prepare_chat_history)['empty_value'])) \
        .replace("Details: \n", "Details:\n")
    completion = "Interviewer: " + separated_observations[i].rsplit("Interviewer: ", maxsplit=1)[-1]
    
    prompt_args = parse(models_metadata['ask_how_it_works'][-1]['prompt_template'], 
                        prompt).named
    completion_args = parse(models_metadata['ask_how_it_works'][-1]['completion_template'], 
                        completion).named
    
    observation_details = dict(model_name="ask_how_it_works", 
                           prompt_template=models_metadata['ask_how_it_works'][-1]['prompt_template'], 
                           completion_template=models_metadata['ask_how_it_works'][-1]['completion_template'], 
                           prompt_args=prompt_args, 
                           completion_args=completion_args,
                           prompt = prompt,
                           completion=completion)
    all_observation_details.append(observation_details)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


### debugging

In [85]:
prompt = separated_observations[28].rsplit("Interviewer: ", maxsplit=1)[0] \
            .replace("\n\n\n", "\nNA\n\n") \
            .replace("Details: \n", "Details:\n")
completion = "Interviewer: " + separated_observations[28].rsplit("Interviewer: ", maxsplit=1)[-1]

In [86]:
prompt

'Current section chat:\nNA\n\nDetails:\nSubject: NA\nQuestion type: how it works\nIs question correct: 0\n'

In [87]:
models_metadata['ask_how_it_works'][-1]['prompt_template']

'Current section chat:\n{current_section_chat}\n\nDetails:\nSubject: {subject}\nQuestion type: {question_type}\nIs question correct: {is_completion_correct}\n'

In [88]:
prompt_args = parse(models_metadata['ask_how_it_works'][-1]['prompt_template'], 
                    prompt)

In [89]:
prompt_args.named

{'current_section_chat': 'NA',
 'subject': 'NA',
 'question_type': 'how it works',
 'is_completion_correct': '0'}

In [64]:
completion

'Interviewer: What are the advantages and disadvantages of using precision as a metric?'

In [73]:
parse("a{b}c{d}", "a\ncd")

<Result () {'b': '\n', 'd': 'd'}>

### upload to ft dataset

In [29]:
for i in range(len(all_observation_details)):
    print(i)
    add_observation_to_raw_finetuning_dataset(all_observation_details[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33


# validate answer how it works

## first set

In [31]:
kshot_validate_answer_how_it_works_prompt = kshot_prompt_templates.validate_answer_how_it_works

kshot_validate_answer_how_it_works_prompt = kshot_validate_answer_how_it_works_prompt.split("Below are some correct examples:")[-1].lstrip()



separated_observations = kshot_validate_answer_how_it_works_prompt.split("Current section chat:")


separated_observations = separated_observations[1:]

for i in range(len(separated_observations)):
    separated_observations[i] = "Current section chat:" + separated_observations[i].rstrip()

separated_observations[-1] = separated_observations[-1][:-6]

separated_observations

In [40]:
len(separated_observations)

30

In [42]:
all_observation_details = []
for i in range(len(separated_observations)):
    print(i)
    prompt = separated_observations[i] \
        .rsplit("Correct Response: ", maxsplit=1)[0] \
        .replace("\n\n\n", "\n{}\n\n".format(get_default_args(prepare_chat_history)['empty_value'])) \
        .replace("Details: \n", "Details:\n")
    completion = "Correct Response: " + separated_observations[i].rsplit("Correct Response: ", maxsplit=1)[-1]
    
    prompt_args = parse(models_metadata['validate_answer_how_it_works'][-1]['prompt_template'], 
                        prompt).named
    completion_args = parse(models_metadata['validate_answer_how_it_works'][-1]['completion_template'], 
                        completion).named
    
    observation_details = dict(model_name="validate_answer_how_it_works", 
                           prompt_template=models_metadata['validate_answer_how_it_works'][-1]['prompt_template'], 
                           completion_template=models_metadata['validate_answer_how_it_works'][-1]['completion_template'], 
                           prompt_args=prompt_args, 
                           completion_args=completion_args,
                           prompt = prompt,
                           completion=completion)
    all_observation_details.append(observation_details)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [44]:
all_observation_details[0]

{'model_name': 'validate_answer_how_it_works',
 'prompt_template': 'Current section chat:\n{current_section_chat}\n\nDetails:\nQuestion type: {question_type}\nIs validation correct: {is_completion_correct}',
 'completion_template': 'Correct Response: {correct_response}\nValidation of response: {validation_of_response}',
 'prompt_args': {'current_section_chat': "Interviewer: How does undersampling the majority class help with class imbalance?\nApplicant: I don't understand",
  'question_type': 'how it works',
  'is_completion_correct': '1\n'},
 'completion_args': {'correct_response': 'By undersampling the majority class, the ratio between the number of observations in the majority and minority classes decreases, making the dataset less imbalanced.',
  'validation_of_response': '0'},
 'prompt': "Current section chat:\nInterviewer: How does undersampling the majority class help with class imbalance?\nApplicant: I don't understand\n\nDetails:\nQuestion type: how it works\nIs validation cor

## set 2

In [60]:
kshot_validate_answer_how_it_works_prompt = kshot_prompt_templates.validate_answer_how_it_works_2
separated_observations = kshot_validate_answer_how_it_works_prompt.split("Current section chat:")
separated_observations = separated_observations[1:]

for i in range(len(separated_observations)):
    separated_observations[i] = "Current section chat:" + separated_observations[i].rstrip()

# separated_observations

In [56]:
len(separated_observations)

22

In [61]:
all_observation_details = []
for i in range(len(separated_observations)):
    print(i)
    prompt = separated_observations[i] \
        .rsplit("Correct Response: ", maxsplit=1)[0] \
        .replace("\n\n\n", "\n{}\n\n".format(get_default_args(prepare_chat_history)['empty_value'])) \
        .replace("Details: \n", "Details:\n")
    completion = "Correct Response: " + separated_observations[i].rsplit("Correct Response: ", maxsplit=1)[-1]
    
    prompt_args = parse(models_metadata['validate_answer_how_it_works'][-1]['prompt_template'], 
                        prompt).named
    completion_args = parse(models_metadata['validate_answer_how_it_works'][-1]['completion_template'], 
                        completion).named
    
    observation_details = dict(model_name="validate_answer_how_it_works", 
                           prompt_template=models_metadata['validate_answer_how_it_works'][-1]['prompt_template'], 
                           completion_template=models_metadata['validate_answer_how_it_works'][-1]['completion_template'], 
                           prompt_args=prompt_args, 
                           completion_args=completion_args,
                           prompt = prompt,
                           completion=completion)
    all_observation_details.append(observation_details)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [63]:
all_observation_details[-1]

{'model_name': 'validate_answer_how_it_works',
 'prompt_template': 'Current section chat:\n{current_section_chat}\n\nDetails:\nQuestion type: {question_type}\nIs validation correct: {is_completion_correct}\n',
 'completion_template': 'Correct Response: {correct_response}\nValidation of response: {validation_of_response}',
 'prompt_args': {'current_section_chat': "Interviewer: How does one-hot encoding work?\nApplicant: Sorry, i don't know.",
  'question_type': 'how it works',
  'is_completion_correct': '1'},
 'completion_args': {'correct_response': 'One-hot encoding converts each unique categorical value to a binary vector.',
  'validation_of_response': '0'},
 'prompt': "Current section chat:\nInterviewer: How does one-hot encoding work?\nApplicant: Sorry, i don't know.\n\nDetails:\nQuestion type: how it works\nIs validation correct: 1\n",
 'completion': 'Correct Response: One-hot encoding converts each unique categorical value to a binary vector.\nValidation of response: 0'}

### upload 

In [64]:
for i in range(len(all_observation_details)):
    print(i)
    add_observation_to_raw_finetuning_dataset(all_observation_details[i])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
